In [1]:
import mysql.connector
import random

# Connect to MySQL Database
try:

    connection = mysql.connector.connect(
        table_name = "patient_data",
        host = input("Enter the host (default is 'localhost'): ") or "localhost",
        user= input("Enter username (default is 'root'): ") or "root",
        password = input("Enter password: "),
        database = 'mysql'
    )
    if connection.is_connected():
        print("Connected to MySQL Database")

        # Create cursor
        cursor = connection.cursor()

        # Query random medical record number from patient_data table
        query_patient_data = "SELECT MedicalRecordNumber FROM patient_data ORDER BY RAND() LIMIT 1"
        cursor.execute(query_patient_data)
        medical_record_number = cursor.fetchone()[0]

        # Query credit card info using the random medical record number
        query_credit_card_data = f"SELECT * FROM credit_card_data WHERE MedicalRecordNumber = '{medical_record_number}'"
        cursor.execute(query_credit_card_data)
        credit_card_info = cursor.fetchone()

        # Query patient address using the random medical record number
        query_patient_address = f"SELECT * FROM patient_address WHERE MedicalRecordNumber = '{medical_record_number}'"
        cursor.execute(query_patient_address)
        patient_address = cursor.fetchone()

        # Print the results
        print("Medical Record Number:", medical_record_number)
        print("Credit Card Info:", credit_card_info)
        print("Patient Address:", patient_address)

except mysql.connector.Error as e:
    print("Error connecting to MySQL Database:", e)

finally:
    # Close connection
    if 'connection' in locals():
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")


Connected to MySQL Database
Error connecting to MySQL Database: 1146 (42S02): Table 'mysql.patient_data' doesn't exist
MySQL connection is closed
